In [85]:
class Element: #格子のオブジェクト
    def __init__(self, position,elev,coeff,width): # コンストラクタ(selfがオブジェクトの個性的な感じ，それ以降の変数は個性の内訳)
        self.position = position # 格子点の位置
        self.elev = elev # 格子点の標高
        self.coeff = coeff  # 粗度係数
        self.width = width #川幅
        self.depth = None  # Noneのやつはあとでdefで決める必要あり
        self.old_depth = None
        self.upnoads = []
        self.dnnoads = []
        self.time_evo  = None

    #数値計算法の選択
    def set_Euler(self):
        self.time_evo = Euler()

    def set_Runge_Kutta_4th(self):
        self.time_evo = Runge_Kutta_4th()

    # 質量保存則
    def solve_mass_equation(self,dt):
        self.time_evo.update_depth(self,dt)   # hの更新
    
    def calc_increment(self):
        fluxin = 0.0
        fluxout = 0.0

        for upnoad in self.upnoads:
            fluxin += upnoad.calc_flux()
        
        for dnnoad in self.dnnoads:
            fluxout += dnnoad.calc_flux()
            
        dh = (fluxin - fluxout)/self.width
        return dh
    

    #def set_upnoad(self,upnoad):
    #    self.upnoad = upnoad

    def set_upnoad(self,upnoad):
        self.upnoads.append(upnoad)
        
    def set_dnnoad(self,dnnoad):
        self.dnnoads.append(dnnoad)
    
    def set_depth(self,value):
        self.depth = value
    
    def set_old_depth(self,value):   # 現時刻の水深
        self.old_depth = value

    def get_variable_depth(self):
        return self.depth

    def get_variable_old_depth(self):
        return self.old_depth
    
    def get_position(self):
        return self.position
    
    def get_elev(self):
        return self.elev
    
    def get_width(self):
        return self.width
    
    def get_coeff(self):
        return self.coeff
    
    def get_courant(self,dt):
        courant = 0
        for upnoad in self.upnoads:
            courant = courant + dt*upnoad.get_variable_q()/(self.width*self.depth+0.0001)  # いったんこれ
            
        return courant
        


class Euler:
    # 水深の時間発展
    def update_depth(self,Element,dt):
        uppdated_depth = Element.get_variable_depth()+Element.calc_increment()*dt
        Element.set_depth(uppdated_depth)





In [86]:
import numpy as np

class Runge_Kutta_4th(object):
    def __init__(self):
        self.stage = 0
        self.hold = None
        self.increments = []

    def update_stage(self):
        if self.stage < 3:
            self.stage += 1
        else:
            self.stage = 0
    
    def set_hold(self,value):
        self.hold = value
    
    def update_stage0_variables(self,Element,dt):
        k1 = Element.calc_increment()
        uppdated_depth = Element.get_variable_depth()+0.5*k1*dt
        Element.set_depth(uppdated_depth)
        self.increments.append(k1)
        self.update_stage()
    
    def update_stage1_variables(self,Element,dt):
        k2 = Element.calc_increment()
        uppdated_depth = Element.get_variable_depth()+0.5*k2*dt
        Element.set_depth(uppdated_depth)
        self.increments.append(k2)
        self.update_stage()
    
    def update_stage2_variables(self,Element,dt):
        k3 = Element.calc_increment()
        uppdated_depth = Element.get_variable_depth()+k3*dt
        Element.set_depth(uppdated_depth)
        self.increments.append(k3)
        self.update_stage()

    def update_stage3_variables(self,Element,dt):
        k4 = Element.calc_increment()
        self.increments.append(k4)
        self.update_stage()

    def update_depth(self,Element,dt):
        if self.stage == 0:
            self.hold = Element.get_variable_depth()
            self.update_stage0_variables(Element,dt)

        elif self.stage==1:
            self.update_stage1_variables(Element,dt)
        
        elif self.stage==2:
            self.update_stage2_variables(Element,dt)

        else:
            self.update_stage3_variables(Element,dt)
            uppdated_depth = self.hold + (1/6)*(self.increments[0]+2*self.increments[1]+2*self.increments[2]+self.increments[3])*dt
            Element.set_depth(uppdated_depth)
            self.increments = []
        



In [87]:
import numpy as np
class Node:
    def __init__(self):
        self.q = None
        self.down_element =None
        self.up_element = None
        

    def set_upelement(self,up_element):
        self.up_element = up_element

    def set_downelement(self,down_element):
        self.down_element = down_element

    def get_upelement(self):
        return self.up_element
    
    def get_downelement(self):
        return self.down_element

    def set_q(self,value):
        self.q = value

    def get_variable_q(self):
        return self.q
    
    # Nodeにおけるフラックス
    def calc_flux(self):
        #dx = abs(self.down_element.get_position()-self.up_element.get_position())
        flux = self.q/150
        return flux
    
    # 運動方程式(拡散波近似)
    def solve_momentum_equation(self):
        Hup     = self.up_element.get_elev() + self.up_element.get_variable_depth()
        Hdn     = self.down_element.get_elev() + self.down_element.get_variable_depth()
        hup     = self.up_element.get_variable_depth()
        hdn     = self.down_element.get_variable_depth()
        Bup = self.up_element.get_width()
        Bdn = self.down_element.get_width()
        
        h = (hup+hdn)/2.0
        B = (Bup+Bdn)/2.0

        h = np.max([0.0,h])    # 水位が負値をとったときの処理
        n = (self.up_element.get_coeff()+self.down_element.get_coeff())/2.0
        dx    = abs(self.down_element.get_position()-self.up_element.get_position()) 
        grad = np.abs((Hup-Hdn)/dx)
        self.q = B*(1/n)*h**(5/3)*np.sqrt(grad)*np.sign((Hup-Hdn)/dx)
        


In [96]:
#from Element import Element
#from Node import Node

import numpy as np
import pandas as pd
import time as timers
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# 以下，メイン文
# 河川の地形の作成
z = np.loadtxt("input/zb_river.txt")
w = np.loadtxt("input/width.txt")
sec = np.loadtxt("input/sec_map.txt")

#本川の地形配列の作成
zb = []
width = []
x = []  # x座標(今は雑)
count = 0
for i in range(sec.shape[0]):
    for j in range(sec.shape[1]):
        if sec[i,j] < 114 and sec[i,j] >= 1:
            zb.append(z[i,j])
            width.append(w[i,j])
            x.append(count*150)  # 150m間隔に格子を配置
            count = count +1
            #print(sec[i,j])

# 支川の地形配列の作成
zb2 = []
width2 = []
x2 = []  # x座標(今は雑)
count = 0
xjunc = 97*150   # secmapで言うところの97で合流していたので，このような処理としている

for i in range(sec.shape[0]):
    for j in range(sec.shape[1]):
        if sec[i,j] < 350 and sec[i,j] >= 329:
            zb2.append(z[i,j])
            #width2.append(w[i,j])
            width2.append(100)
            x2.append(xjunc + count*150)  # 合流点から見た河口からの距離
            count = count +1



# 地形配列
zb = np.array(zb)
x = np.array(x)
width = np.array(width)

zb2 = np.array(zb2)
x2 = np.array(x2)
width2 = np.array(width2)

# 直線流路化
#zb = np.linspace(zb[0], zb[-1], len(zb))

zb = zb[::-1]
x  = x[::-1]
width = width[::-1]

zb2 = zb2[::-1]
x2  = x2[::-1]
width2 = width2[::-1]


ib = (zb[-1]-zb[0])/(x[-1] - x[0] )

# 流量データの読み込み
init_dt = 1
dt = init_dt
df = pd.read_csv("input/Boundary2.csv")
Qb = list(df[df.keys()[1]])
Qb2 = list(df[df.keys()[2]])
Qb = np.array(Qb)
Qb2 = np.array(Qb2)
maxt = Qb.shape[0]
np.savetxt("input/zb.csv",zb, fmt="%.6f", delimiter=",")
np.savetxt("input/zb2.csv",zb2, fmt="%.6f", delimiter=",")
# print(x)
# print(zb)
# print(width)
# print(x2)
# print(zb2)
# print(width2)

# 河川のエレメント＆ノードオブジェクトへの属性の指定
elements = [Element(x[i],zb[i],0.03,width[i]) for i in range(zb.shape[0])]
nodes = [Node() for i in range(zb.shape[0])]
n_riv = len(elements)

elements2 = [Element(x2[i],zb2[i],0.03,width2[i]) for i in range(zb2.shape[0])]
nodes2 = [Node() for i in range(zb2.shape[0])]
n_riv2 = len(elements2)

# エレメント＆ノードの接続(本川)
for i in range(len(elements)):
    nodes[i].set_upelement(elements[i])
    
for i in range(1,len(elements)):
    elements[i].set_upnoad(nodes[i-1])

for i in range(len(elements)-1):
    nodes[i].set_downelement(elements[i+1])
    
for i in range(len(elements)):
    elements[i].set_dnnoad(nodes[i])

# エレメント＆ノードの接続(支川)
for i in range(len(elements2)):
    nodes2[i].set_upelement(elements2[i])
    
for i in range(1,len(elements2)):
    elements2[i].set_upnoad(nodes2[i-1])

for i in range(len(elements2)-1):
    nodes2[i].set_downelement(elements2[i+1])
    
for i in range(len(elements2)):
    elements2[i].set_dnnoad(nodes2[i])


# 境界条件(本川)
bc_upnode = Node()
elements[0].set_upnoad(bc_upnode)
bc_dnelement = Element(x[n_riv-1]-150,zb[n_riv-1]-ib*150,0.03,width[n_riv-1])
nodes[n_riv-1].set_downelement(bc_dnelement)

# 境界条件(支川)
bc_upnode2 = Node()
elements2[0].set_upnoad(bc_upnode2)

#支川と本川の接続 # 96番目のエレメントが接続部
nodes2[-1].set_downelement(elements[16])   
elements[16].set_upnoad(nodes2[-1])

for target_element in elements:  #本川
    #target_element.set_Euler()
    target_element.set_Runge_Kutta_4th()

for target_element in elements2: #本川
    #target_element.set_Euler()
    target_element.set_Runge_Kutta_4th()

# 初期条件
for i in range(n_riv): #本川
    elements[i].set_depth(0)
    nodes[i].set_q(0)

for i in range(n_riv2): #本川
    elements2[i].set_depth(0)
    nodes2[i].set_q(0)



Hs = []
Qs = []
Hs2 = []
Qs2 = []

time = 0
t = 0
# select max time[h]
# maxt = 314
maxt = 10

### check variables ###
total_Qin = 0
total_water = 0
total_Qout = 0

# 計算開始時刻を記録
start_time = timers.time()
#####################
### 時間発展ループ ###
#####################
while time/3600.0 < maxt:
    H = []  # 水深の縦断分布
    Q = []  # 流量の縦断分布
    H2 = []
    Q2 = []
    
    H.append(time/3600)
    Q.append(time/3600)
    H2.append(time/3600)
    Q2.append(time/3600)
    
    # boundary contion
    bc_upnode.set_q(Qb[int(time // 3600)])
    bc_upnode2.set_q(Qb2[int(time // 3600)])
    bc_dnelement.set_depth(elements[n_riv-1].get_variable_depth())


    # Iwasaki 修正 ルンゲクッタ 各ステージの流出する流量(q)のリスト作成
    qout_stages = []
    for stage in range(4):
        # Iwasaki 修正 ルンゲクッタ 各ステージの流出する流量(q)の追加
        qout_stages.append(elements[-1].dnnoads[0].get_variable_q())

        for target_element in elements:
            target_element.solve_mass_equation(dt)# 質量保存側
            
        for target_element in elements2:
            target_element.solve_mass_equation(dt)# 質量保存側
            
        for target_node in nodes:
            target_node.solve_momentum_equation()
            
        for target_node in nodes2:
            target_node.solve_momentum_equation()

    total_Qin += Qb[int(time // 3600)] * dt
    total_Qin += Qb2[int(time // 3600)] * dt

    # Iwasaki 修正 ルンゲクッタ　 各ステージの重みを考慮して, この時間ステップ内で流出した流量をTotalに追加
    total_Qout += (qout_stages[0] + 2*qout_stages[1] + 2*qout_stages[2] + qout_stages[3])*dt/6

    total_water = 0
    for target_element in elements:
        h = target_element.get_variable_depth()
        H.append(h)
        total_water += h*target_element.get_width()*150
        
    for target_element in elements2:
        h2 = target_element.get_variable_depth()
        H2.append(h2)
        total_water += h2*target_element.get_width()*150

    for target_node in nodes:
        q = target_node.get_variable_q()
        Q.append(q)
    
    for target_node in nodes2:
        q2 = target_node.get_variable_q()
        Q2.append(q2)

    # dtの判定
    H = np.array(H)
    Q = np.array(Q)

    # dtの制御
    #courants = []
    #if (time>3600):
    #    for target_element in elements:
    #        courant = target_element.get_courant(dt)
    #        courants.append(courant)
            #print(courant,target_element.get_variable_depth())

    #    if max(courants)>150:
    #        dt = 150/(max(courants)/dt)
    #    else:
    #        dt = init_dt
            


    if Qb[int(time // 3600)]>200:
        dt = 2.5
    else:
        dt = init_dt
    


    #dt = init_dt
    t = t+1
    time = time+dt

    if np.mod(time/3600,1)==0:
        # check total mass of water
        print("total_Qin:",total_Qin," total_water:",total_water," total_Qout:",total_Qout,"sum:",total_Qout+total_water-total_Qin)
        print("numerical error = ",(total_Qout+total_water-total_Qin)/total_Qin)
        Hs.append(H)
        Qs.append(Q)
        Hs2.append(H2)
        Qs2.append(Q2)
        print("time:",time/3600,"  [h]",r"Q(m^3/s):","dt:",dt,"Qup:",Qb[int(time // 3600)])
# """
# 計算終了時刻を記録
end_time = timers.time()
# 経過時間を表示
print(f"計算時間: {end_time - start_time:.3f} 秒")

# save
Hs = np.array(Hs)
Qs = np.array(Qs)

# 水深⇒zbの変換
for i in range(zb.shape[0]):
    Hs[:,i+1] = Hs[:,i+1]+zb[i]

np.savetxt("./out/Hs.csv",Hs, fmt="%.6f", delimiter=",")
np.savetxt("./out/Qs.csv",Qs, fmt="%.6f", delimiter=",")

Hss = Hs[:,1:]

fig, ax = plt.subplots(figsize=(11,4))  # 縦横同じくらいに

line_zb, = ax.plot([], [], color="saddlebrown", linewidth=2, label="bed elevation")
line_H,  = ax.plot([], [], marker="o", color="blue", linewidth=2, label="water level")

ax.set_xlim(x.min(), x.max())
ax.set_xlabel(" L [m]")
ax.set_ylabel("H [m]")
ax.legend(loc="best")

# 縦軸を毎フレーム更新して見やすくする
def update(frame):
    h = Hss[frame,:]
    H =  h
    line_zb.set_data(x, zb)
    line_H.set_data(x, H)
    ymin = min(zb.min(), H.min()) - 0.5
    ymax = H.max() + 0.5
    ax.set_ylim(-11, 15)
    ax.set_title(f" t = {Hs[frame,0]:.1f} h")
    return line_zb, line_H

ani = animation.FuncAnimation(fig, update, frames=Hss.shape[0], blit=True)

ani.save("./out/waterlevel.gif", writer="pillow", fps=20)
plt.close()

Qss = Qs[:,1:]   # 流量データ（時系列 × 空間）

fig, ax = plt.subplots(figsize=(11,4))

line_Q, = ax.plot([], [], marker="o", color="red", linewidth=2, label=" Q")

ax.set_xlim(x.min(), x.max())
ax.set_xlabel(" L [m]")
ax.set_ylabel(r"Q [m^3/s]")   # 単位は適宜
ax.legend(loc="best")

def update(frame):
    Q = Qss[frame,:]
    line_Q.set_data(x, Q)
    ymin = Q.min() - 0.5
    ymax = Q.max() + 0.5
    ax.set_ylim(0, 2700)
    ax.set_title(f" t = {Hs[frame,0]:.1f} h")
    return line_Q,

ani = animation.FuncAnimation(fig, update, frames=Qss.shape[0], blit=True)

ani.save("./out/discharge.gif", writer="pillow", fps=20)
plt.close()
np.savetxt("./out/Qs.csv",Qs, fmt="%.6f", delimiter=",")

# """

KeyboardInterrupt: 